# Cleanup All Resources

This notebook cleans up all AWS resources created across Labs 1-4.

**What Gets Deleted:**
- 4 Knowledge Bases and their S3 buckets
- 2 DynamoDB tables
- 1 Lambda function and IAM role
- 1 AgentCore Gateway and targets
- 1 Cognito User Pool
- SSM parameters
- Any AgentCore Runtimes created in labs

**Warning:** This action cannot be undone. Make sure you've saved any data you need before proceeding.

### Initialize Environment

In [ ]:
import sys
import boto3
import json
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent / 'helper'))
from display_helper import print_success, print_info
from agentcore_helper import AgentCoreHelper
from lambda_helper import LambdaHelper
from cognito_helper import CognitoHelper

boto_session = boto3.session.Session()
region = boto_session.region_name
account_id = boto3.client('sts').get_caller_identity()['Account']
cfn_client = boto3.client('cloudformation', region_name=region)

%store -r stack_name
%store -r base_name

print_info(f"Region: {region}")
print_info(f"Account: {account_id}")
print_info(f"Stack name: {stack_name}")
print_info(f"Base name: {base_name}")

### Remove deployed stack

In [ ]:
# WARNING: This will delete ALL resources created by the stack
# Uncomment the lines below to run cleanup

print("Starting comprehensive cleanup...")
print(f"Stack: {stack_name}\n")

# Step 1: Empty S3 data buckets
print("Step 1: Emptying S3 data buckets...")
s3_client = boto3.client('s3', region_name=region)
data_buckets = [
    f'{base_name}-sports-data-{account_id}',
    f'{base_name}-compliance-data-{account_id}',
    f'{base_name}-news-data-{account_id}',
    f'{base_name}-films-data-{account_id}'
]

for bucket_name in data_buckets:
    try:
        # List and delete all objects
        paginator = s3_client.get_paginator('list_objects_v2')
        for page in paginator.paginate(Bucket=bucket_name):
            if 'Contents' in page:
                objects = [{'Key': obj['Key']} for obj in page['Contents']]
                s3_client.delete_objects(Bucket=bucket_name, Delete={'Objects': objects})
        print(f"  ✓ Emptied: {bucket_name}")
    except Exception as e:
        print(f"  Note: {bucket_name} - {e}")

# Step 2: Delete S3 Vector buckets
print("\nStep 2: Deleting S3 Vector buckets...")
s3vectors_client = boto3.client('s3vectors', region_name=region)
vector_buckets = [
    f'{base_name}-sports-vector-{account_id}',
    f'{base_name}-compliance-vector-{account_id}',
    f'{base_name}-news-vector-{account_id}',
    f'{base_name}-films-vector-{account_id}'
]

for bucket_name in vector_buckets:
    try:
        # List and delete all indexes first
        indexes = s3vectors_client.list_indexes(vectorBucketName=bucket_name)
        for index in indexes.get('indexes', []):
            s3vectors_client.delete_index(
                vectorBucketName=bucket_name,
                indexName=index['indexName']
            )
        # Delete the vector bucket
        s3vectors_client.delete_vector_bucket(vectorBucketName=bucket_name)
        print(f"  ✓ Deleted: {bucket_name}")
    except Exception as e:
        print(f"  Note: {bucket_name} - {e}")

# Step 3: Delete SSM parameters
print("\nStep 3: Deleting SSM parameters...")
ssm_client = boto3.client('ssm', region_name=region)
ssm_params = [
    '/sports-agent/gateway_url',
    '/sports-agent/gateway_id',
    '/sports-agent/cognito_user_pool_id',
    '/sports-agent/cognito_domain',
    '/sports-agent/cognito_client_id',
    '/sports-agent/cognito_client_secret',
    '/sports-agent/cognito_discovery_url'
]

for param_name in ssm_params:
    try:
        ssm_client.delete_parameter(Name=param_name)
        print(f"  ✓ Deleted: {param_name}")
    except ssm_client.exceptions.ParameterNotFound:
        pass
    except Exception as e:
        print(f"  Note: {param_name} - {e}")

# Step 4: Delete CloudFormation stack
print("\nStep 4: Deleting CloudFormation stack...")
print("This will delete Gateway, Knowledge Bases, DynamoDB tables, Lambda, etc.")
cfn_client.delete_stack(StackName=stack_name)
print("Stack deletion initiated. Waiting for completion (10-15 minutes)...")

cfn_client.get_waiter('stack_delete_complete').wait(StackName=stack_name)

print("\n✅ Complete cleanup finished!")
print("All resources including S3 buckets, vector buckets, and SSM parameters have been removed.")

### Clean Up Lab 2 Resources

Delete resources created specifically in Lab 2 (Sports Agent with Gateway).

**Resources Deleted:**
- Lambda function and IAM role (if created in Lab 2)
- AgentCore Gateway and IAM role
- Cognito User Pool (Lab 2 specific)

In [ ]:
%store -r lab_gateway_id
%store -r lab_user_pool_id
%store -r lab_lambda_function_name
%store -r lab_lambda_role_name
%store -r lab_gateway_name

In [ ]:

if lab_lambda_function_name:
    # Delete Lambda and its role
    print_info("\nDeleting Lambda function and role...")
    # Initialize Lambda helper
    lambda_helper = LambdaHelper()
    lambda_helper.delete_lambda_and_role(
        function_name=lab_lambda_function_name,
        role_name=lab_lambda_role_name
    )

if lab_gateway_name:
    # Delete Gateway IAM role
    print_info("\nDeleting Gateway IAM role...")
    try:
        iam_client = boto3.client('iam')
        gateway_role_name = f'agentcore-{lab_gateway_name}-role'
        policies = iam_client.list_role_policies(RoleName=gateway_role_name)
        for policy_name in policies['PolicyNames']:
            iam_client.delete_role_policy(RoleName=gateway_role_name, PolicyName=policy_name)
        iam_client.delete_role(RoleName=gateway_role_name)
        print_success(f"   Deleted role: {gateway_role_name}")
    except Exception as e:
        print_info(f"   Note: {e}")

if lab_user_pool_id:
    # Initialize Cognito helper
    cognito_helper = CognitoHelper()
    # Delete Cognito User Pool
    print_info("\nDeleting Cognito User Pool...")
    cognito_helper.delete_user_pool(lab_user_pool_id)

if lab_gateway_id:

    # Initialize AgentCore helper
    agentcore_helper = AgentCoreHelper()
    print("\nDeleting Gateway...")
    agentcore_helper.delete_gateway(lab_gateway_id)

### Clean Up Lab 3 Resources

Delete resources created in Lab 3 (Sports Agent on Runtime).

**Resources Deleted:**
- AgentCore Runtime and ECR repository
- Runtime execution IAM role and policy
- SSM parameter for gateway URL

In [ ]:
%store -r lab_agent_arn
%store -r lab_agent_execution_role_name
%store -r lab_agent_execution_policy_name
%store -r lab_ssm_parameter

In [ ]:
if lab_agent_arn:
    # Initialize AgentCore helper
    agentcore_helper = AgentCoreHelper()
    agentcore_helper.runtime_resource_cleanup(runtime_arn=lab_agent_arn)
    print_success("✅ Agentcore Runtime deleted")

if lab_agent_execution_role_name:
    # Delete the runtime execution role and policy
    agentcore_helper.delete_agentcore_runtime_execution_role(
        role_name=lab_agent_execution_role_name,
        policy_name=lab_agent_execution_policy_name
    )

    print_success("✅ IAM role and policy deleted")

if lab_ssm_parameter:
    # Delete SSM parameter
    try:
        # Initialize AWS clients
        ssm_client = boto3.client('ssm')
        ssm_client.delete_parameter(Name=lab_ssm_parameter)
        print_success(f"✅ Deleted SSM parameter: {lab_ssm_parameter}")
    except ssm_client.exceptions.ParameterNotFound:
        print("SSM parameter already deleted or doesn't exist")
    except Exception as e:
        print(f"Note: {e}")

### Clear Stored Variables

Clear all IPython stored variables used across the lab notebooks.

In [ ]:
# Clear all stored variables
%store -z

print_success("All stored variables cleared")

### ✅ Cleanup Complete!